In [1]:
import numpy as np
from numpy import genfromtxt
from pathlib import Path
import math
from scipy.signal import savgol_filter

In [2]:
#  Functions

def eulers_2_rot_matrix(x):
    """
    EULER_2_ROT_MATRIX transforms a set of euler angles into a rotation  matrix 
    input vector of euler angles 
    [gamma_x, beta_y, alpha_z]  are ZYX Eulers angles in radians
    """
    gamma_x=x[0];beta_y=x[1];alpha_z=x[2];
    R = rotz(alpha_z) * roty(beta_y) * rotx(gamma_x);
    return R

def rotx(t):
    # ROTX Rotation about X axis
    ct = math.cos(t);
    st = math.sin(t);
    r =  np.matrix([[1,	0,	0],
                    [0,	ct,	-st],
                    [0,	st,	ct]]);
    return r

def roty(t):
    # ROTY Rotation about Y axis
    ct = math.cos(t);
    st = math.sin(t);
    r =  np.matrix([[ct,	0,	st],
                    [0,	1,	0],
                    [-st	,0,	ct]]);
    return r

def rotz(t):
    # ROTZ Rotation about Z axis
	ct = math.cos(t);
	st = math.sin(t);
	r = np.matrix([[ct,	-st,	0],
                   [st,	ct,	0],
                   [0,	0,	1]]);
    


In [3]:
# KINECT - Filtering the raw data

# Position directory setup
correct_dir_pos_orig = "Segmented Movements/Kinect/Positions"
incorrect_dir_pos_orig = "Incorrect Segmented Movements/Kinect/Positions"
correct_dir_ang_orig = "Segmented Movements/Kinect/Angles"
incorrect_dir_ang_orig = "Incorrect Segmented Movements/Kinect/Angles"

path_to_correct_pos = Path(correct_dir_pos_orig)
path_to_incorrect_pos = Path(incorrect_dir_pos_orig)
path_to_correct_ang = Path(correct_dir_ang_orig)
path_to_incorrect_ang = Path(incorrect_dir_ang_orig)
paths_original = [path_to_correct_pos, path_to_incorrect_pos, path_to_correct_ang, path_to_incorrect_ang]

# Angles directory setup
correct_dir_pos_filt = "Filtered Segmented Movements/Kinect/Positions"
incorrect_dir_pos_filt = "Filtered Incorrect Segmented Movements/Kinect/Positions"
correct_dir_ang_filt = "Filtered Segmented Movements/Kinect/Angles"
incorrect_dir_ang_filt = "Filtered Incorrect Segmented Movements/Kinect/Angles"

# Create Path objects
path_to_correct_filtered_pos = Path(correct_dir_pos_filt)
path_to_incorrect_filtered_pos = Path(incorrect_dir_pos_filt)
path_to_correct_filtered_ang = Path(correct_dir_ang_filt)
path_to_incorrect_filtered_ang = Path(incorrect_dir_ang_filt)

# Create directories
path_to_correct_filtered_pos.mkdir(parents=True, exist_ok=True)
path_to_incorrect_filtered_pos.mkdir(parents=True, exist_ok=True)
path_to_correct_filtered_ang.mkdir(parents=True, exist_ok=True)
path_to_incorrect_filtered_ang.mkdir(parents=True, exist_ok=True)

for directory in paths_original:
    for file_path in directory.iterdir():
        if file_path.is_file():
            read_data = genfromtxt(file_path, delimiter=',')        
            # Work through 3 columns at a time (one joint)
            for i in range(0, read_data.shape[1], 3):
                joint_coordinate_slice = read_data[:, i:i+3]
                filtered_joint = joint_coordinate_slice.copy()

                # Use Savitzky-Golay Smoothing filter on each dimension separately
                # Window length = 5
                # Polynomial order = 2
                for j in range(0, 3):
                    dimension_slice = filtered_joint[:, j]
                    # apply filter Savitzky-Golay filter to the dimension of joint across all time points
                    filtered_data = savgol_filter(dimension_slice, window_length=5, polyorder=2)
                    read_data[:, i+j] = filtered_data # Save new positional data

            smooth_file = "smooth_" + file_path.name
            # Save the filtered data as correct or incorrect in new directories
            if "angle" in file_path.name:
                if "Incorrect" in str(directory.parent):
                    np.savetxt(f'{incorrect_dir_ang_filt}/{smooth_file}', read_data, delimiter=',', fmt='%f')
                else:
                    np.savetxt(f'{correct_dir_ang_filt}/{smooth_file}', read_data, delimiter=',', fmt='%f')
            else:
                if "Incorrect" in str(directory.parent):
                    np.savetxt(f'{incorrect_dir_pos_filt}/{smooth_file}', read_data, delimiter=',', fmt='%f')
                else:
                    np.savetxt(f'{correct_dir_pos_filt}/{smooth_file}', read_data, delimiter=',', fmt='%f')

In [4]:
# VICON - Filtering the raw data

# Position directory setup
correct_dir_pos_orig = "Segmented Movements/Vicon/Positions"
incorrect_dir_pos_orig = "Incorrect Segmented Movements/Vicon/Positions"
correct_dir_ang_orig = "Segmented Movements/Vicon/Angles"
incorrect_dir_ang_orig = "Incorrect Segmented Movements/Vicon/Angles"

path_to_correct_pos = Path(correct_dir_pos_orig)
path_to_incorrect_pos = Path(incorrect_dir_pos_orig)
path_to_correct_ang = Path(correct_dir_ang_orig)
path_to_incorrect_ang = Path(incorrect_dir_ang_orig)
paths_original = [path_to_correct_pos, path_to_incorrect_pos, path_to_correct_ang, path_to_incorrect_ang]

# Angles directory setup
correct_dir_pos_filt = "Filtered Segmented Movements/Vicon/Positions"
incorrect_dir_pos_filt = "Filtered Incorrect Segmented Movements/Vicon/Positions"
correct_dir_ang_filt = "Filtered Segmented Movements/Vicon/Angles"
incorrect_dir_ang_filt = "Filtered Incorrect Segmented Movements/Vicon/Angles"

# Create Path objects
path_to_correct_filtered_pos = Path(correct_dir_pos_filt)
path_to_incorrect_filtered_pos = Path(incorrect_dir_pos_filt)
path_to_correct_filtered_ang = Path(correct_dir_ang_filt)
path_to_incorrect_filtered_ang = Path(incorrect_dir_ang_filt)

# Create directories
path_to_correct_filtered_pos.mkdir(parents=True, exist_ok=True)
path_to_incorrect_filtered_pos.mkdir(parents=True, exist_ok=True)
path_to_correct_filtered_ang.mkdir(parents=True, exist_ok=True)
path_to_incorrect_filtered_ang.mkdir(parents=True, exist_ok=True)

for directory in paths_original:
    for file_path in directory.iterdir():
        if file_path.is_file():
            read_data = genfromtxt(file_path, delimiter=',')        
            # Work through 3 columns at a time (one joint)
            # Avoids case like m10_s05_e09_positions_inc.txt, (117,) in Vicon dataset
            if (len(read_data.shape) != 2):
                print('read_data.shape for file: ', f'{file_path.name}, {read_data.shape}');
                continue;
            for i in range(0, read_data.shape[1], 3):
                joint_coordinate_slice = read_data[:, i:i+3]
                filtered_joint = joint_coordinate_slice.copy()

                # Use Savitzky-Golay Smoothing filter on each dimension separately
                # Window length = 5
                # Polynomial order = 2
                for j in range(0, 3):
                    dimension_slice = filtered_joint[:, j]
                    # apply filter Savitzky-Golay filter to the dimension of joint across all time points
                    filtered_data = savgol_filter(dimension_slice, window_length=5, polyorder=2)
                    read_data[:, i+j] = filtered_data # Save new positional data

            smooth_file = "smooth_" + file_path.name
            # Save the filtered data as correct or incorrect in new directories
            if "angle" in file_path.name:
                if "Incorrect" in str(directory.parent):
                    np.savetxt(f'{incorrect_dir_ang_filt}/{smooth_file}', read_data, delimiter=',', fmt='%f')
                else:
                    np.savetxt(f'{correct_dir_ang_filt}/{smooth_file}', read_data, delimiter=',', fmt='%f')
            else:
                if "Incorrect" in str(directory.parent):
                    np.savetxt(f'{incorrect_dir_pos_filt}/{smooth_file}', read_data, delimiter=',', fmt='%f')
                else:
                    np.savetxt(f'{correct_dir_pos_filt}/{smooth_file}', read_data, delimiter=',', fmt='%f')

read_data.shape for file:  m10_s05_e09_positions_inc.txt, (117,)
read_data.shape for file:  m10_s05_e09_angles_inc.txt, (117,)
